# Рецензии на IMDB

Имеются 25000 рецензий пользователей imdb с бинарными метками, посчитанными по оценкам: 0 при оценке < 5 и 1 при оценке >=7.

Полные данные: https://www.kaggle.com/c/word2vec-nlp-tutorial/data


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

In [2]:
%matplotlib inline

1. **Загружаем данные**

In [3]:
imdb=pd.read_csv('labeledTrainData.tsv', delimiter='\t')
imdb.shape

(25000, 3)

In [4]:
imdb.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
# проверим на сколько сбалансированы классы по sentiment
imdb.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

# 2. **Разбиваем выборку  на обучение и контроль по рецензиям**

попробуем спрогнозировать насколько фильм понравится или нет анализируя рецензии

In [6]:
text_train, text_test, y_train, y_test = train_test_split(imdb.review.values, imdb.sentiment.values)

# векторизируем тексты рецензий с помощью TFIDF т.е. для каждого 
# слова он выделяет отдельный признак
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(sublinear_tf=True, use_idf=True)
X_train=vect.fit_transform(text_train)
X_test=vect.transform(text_test)


# 3.Логистическая регрессия

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

clf = LogisticRegression()
clf.fit(X_train, y_train)
print metrics.accuracy_score(y_test, clf.predict(X_test))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])


0.88752
0.957983503981


точность неплохая но признаков очень много(66 633). Попробуем сделать модель проще для чего используем Лассо

In [8]:
clf=LogisticRegression(C=0.15, penalty='l1')

clf.fit(X_train, y_train)

print np.sum(np.abs(clf.coef_)>1e-4)
print metrics.accuracy_score(y_test, clf.predict(X_test))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

105
0.80368
0.891340557857


при уменьшении колва признаков до 100 упала точность. Пробуем другой способ **Stability Selection**. Он заключается в том что мы много раз делаем Лассо с одинаковыми параметрами регуляризации на  на случайных подвыборках наших исходных данных. Затем для каждого признака делаем score который тем больше чем чаще в Лассо он отбирался как полезный.

In [9]:
from sklearn.linear_model import RandomizedLogisticRegression

rlg=RandomizedLogisticRegression(C=0.13)
rlg.fit(X_train, y_train)

C:\Users\semen\Anaconda2\lib\site-packages\sklearn\linear_model\randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
C:\Users\semen\Anaconda2\lib\site-packages\sklearn\linear_model\randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
C:\Users\semen\Anaconda2\lib\site-packages\sklearn\linear_model\randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
C:\Users\semen\Anaconda2\lib\site-packages\sklearn\linear_model\randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
C:\Users\semen\Anaconda2\lib\site-packages\sklearn\linear_model\randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ 

RandomizedLogisticRegression(C=0.13, fit_intercept=True,
               memory=Memory(cachedir=None), n_jobs=1, n_resampling=200,
               normalize=True, pre_dispatch='3*n_jobs', random_state=None,
               sample_fraction=0.75, scaling=0.5, selection_threshold=0.25,
               tol=0.001, verbose=False)

In [13]:
# посмотрим сколько полезных признаков остается теперь
np.sum(rlg.scores_>0)

117

In [14]:
#настроим для отобранных признаков регрессию
X_train_lasso=X_train[:, rlg.scores_>0]
X_test_lasso=X_test[:, rlg.scores_>0]


clf=LogisticRegression(C=1)
clf.fit(X_train_lasso, y_train)

print metrics.accuracy_score(y_test, clf.predict(X_test_lasso))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_lasso)[:, 1])

0.82016
0.908590185533


итак мы улучшилирезультат но можно еще лучше.

# 4.Метод главных компонент

Создадим 100 новых синтетических признаков на основании 66 633 признаков при помощи Метода главных компонент и на новых признаках построим логистическую регрессию

In [17]:
from sklearn.decomposition import TruncatedSVD

tsvd= TruncatedSVD(n_components=100)
X_train_pea=tsvd.fit_transform(X_train)
X_test_pea=tsvd.transform(X_test)

#Обучим на них логистическую регрессию
clf = LogisticRegression()
clf.fit(X_train_pea, y_train)

print metrics.accuracy_score(y_test, clf.predict(X_test_pea))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pea)[:, 1])

0.85936
0.938413134626


Получаем еще большее улучшение качества в сравнении с обычным отбором признаков

# 5. Случайный лес

На этих же синтетических новых признаках обучим случайный лес

In [19]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train_pea, y_train)

print metrics.accuracy_score(y_test, clf.predict(X_test_pea))
print metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pea)[:, 1])

0.82672
0.908366755639


Как видим качество на случайном лесе получается хуже логистической регрессии. Это происходит потому что признаки которые дает **метод главных компонент** оптимальны именно для линейных классификаторов